# Apertura de archivos

Es posible leer / grabar archivos de texto en Python.
<BR>
* Los datos se almacenan como cadenas de caracteres, en diferentes formatos:  
* separados por punto y coma, con columnas de ancho fijo, etc.
* Pueden ser creados, visualizados o modificados por cualquier editor de texto: ○ Bloc de notas de Windows; Notepad++; Visual Studio Code; etc.
* No tienen formato: texto plano.
* Cada línea o renglón constituye un registro.
* La longitud es variable.
* Utiliza un delimitador, \n por ejemplo para indicar salto de linea.

Sintáxis para la apertura de archivos:
<BR>
<BR>
`variable = open(file, mode)`
<BR>
* `variable`: nombre de la variable que contendrá la referencia al archivo de texto.
* `open`: función que retorna un objeto de tipo Archivo.
* `file`: indica el nombre del archivo de texto, puede contener la ruta donde se encuentra. Cuidar la secuencia de escape por el uso de `\`.
* `mode`: indica el modo de apertura del archivo, por default es modo lectura de un archivo de texto.


## Modos admitidos en open()
* "r" Leer - No crea el archivo si no existe / No borra contenido previo
* "w" Escribir (sobrescribe) - Crea el archivo si no existe / Borra contenido previo
* "a" Agregar (append) -  Crea el archivo si no existe / No borra contenido previo
* "r+" Leer y escribir (sin borrar) - No crea el archivo si no existe / No borra contenido previo
* "w+ " Leer y escribir (borrando el contenido) - Crea el archivo si no existe / Borra contenido previo
* "a+" Leer y escribir (agregando al final) -  Crea el archivo si no existe / No borra contenido previo

## Manejo de rutas
Si no se incluye ruta al archivo, se busca en la misma carpeta donde se encuentra el programa.
<BR>
En esta instrucción, la ruta es inválida: `file = open("data\notas.txt", "r")`, dado el salto de línea ("\n") incluido en ella. Para evitar este error tenemos 3 posibilidades:
* Usar doble barra invertida: La doble contrabarra es también una secuencia de escape que representa a una sola barra invertida:
<BR>`file = open("data\\notas.txt", "r")`
* Usar una sola barra normal: 
<BR>
`file = open("data/notas.txt", "r")`
* Declarar la cadena como cruda: es una cadena de texto en la que los caracteres de escape no se interpretan como caracteres especiales, sino que se tratan como caracteres literales:
<BR>`file = open(r"data\notas.txt", "r")`


In [2]:
# Abrir el archivo data/notas.txt
file = open(r"data\notas.txt", "r")
contenido = file.read()
print(contenido)

Alumno: Peter
Nota: 9



# Cierre de archivos

Veamos porque es una buena práctica cerrar los archivos luego de acceder a ellos:
<BR>
Cuando abrís un archivo, usas recursos del sistema, el OS reserva:
* Un descriptor de archivo (file handle)
* Un bloque de memoria para el buffer de lectura/escritura
* Y un enlace activo entre tu programa y el archivo físico en disco.

Si no lo cerrás, esos recursos quedan ocupados hasta que el programa termina — o hasta que el recolector de basura los libere (lo que no siempre pasa enseguida).
<BR>
Si no cerramos el archivo, podrían darse las siguientes situaciones:
* Bloqueo de archivos: En algunos sistemas (Windows especialmente), el archivo queda bloqueado: no podés borrarlo o abrirlo desde otro programa.
* Memory Leak: Si abrís muchos archivos sin cerrar, el programa usa más memoria y puede quedarse sin descriptores.
* Datos no guardados: En modo escritura ("w" o "a"), el contenido se guarda primero en un buffer. Si no se cierra el archivo, puede que el buffer nunca se vacíe y los datos no se escriban en disco.
* Comportamientos impredecibles: Algunas funciones que esperan que el archivo esté cerrado (como os.remove() o shutil.move()) pueden fallar.

Para cerrar el archivo usamos el método `close`
<BR>
* Vacía los buffers (garantizando que todo lo que escribiste se guarde).
* Libera el descriptor de archivo.
* Rompe el vínculo entre el archivo y tu programa.


In [3]:
# Abrir el archivo data/notas.txt
file = open(r"data\notas.txt", "r")
contenido = file.read()
print(contenido)

# Cerrar el archivo data/notas.txt
file.close()


Alumno: Peter
Nota: 9



# Manejo de excepciones con archivos

Si Python no puede abrir o cerrar el archivo, o algunas de las operaciones sobre el archivo no pueden realizarse se generarán excepciones que interrumpen el flujo normal del programa. Veamos como aplicar el bloque try/except/finally para manejar futuros errores.

In [14]:
# Mejoramos la apertura y el cierre del archivo con try/except/finally
try:
    file = open("data/notas.txt", "r")
    contenido = file.read()
    print(contenido)
except Exception as e:
    print(f"Error: {e}")
finally:
    file.close()

Alumno: Peter
Nota: 9



Si bien poner la instrucción file.close() en el finallly es lo mas razonable, ya que es una acción que siempre debe realizarse, en caso que el archivo no pueda abrirse, la acción file.close() generará una exception. Analicemos algunas maneras de evitarlo:

In [ ]:
import os
print(os.getcwd())

In [28]:
file = None  # inicializamos la variable

try:
    file = open(r"data\notas.txt", "r", encoding="utf-8")
    contenido = file.read()
    print(contenido)
except Exception as e:
    print(f"Error Try: {e}")
finally:
    if file is not None:
        file.close()


Alumno: Peter
Nota: 9



In [35]:
# O podemos incluso envolver file.close() en un bloque try/except
try:
    file = open(r"data\notas.txt", "r")
    contenido = file.read()
    print(contenido)
except Exception as e:
    print(f"Error Try: {e}")
finally:
    try:
        file.close()
    except Exception as e:
        print(f"Error Finally: {e}")

Alumno: Peter 
Nota: 9



# Introducción a `with`

Si ocurre un error entre el open() y el close(), por ejemplo un ValueError, el close() nunca se ejecuta y el archivo queda abierto en memoria.
Aunque podemos usar try / finally para forzar el cierre, vimos que este patrón funciona, pero es largo, repetitivo y propenso a errores si se olvida el finally.

## Sintaxis de `with`

In [36]:
with open("data/notas.txt", "r", encoding="utf-8") as file:
    contenido = file.read()
    print(contenido)

Alumno: Peter 
Nota: 9



¿Qué hace Python internamente?

* Llama a open("data/notas.txt, "r") y guarda el archivo en la variable file
* Ejecuta el bloque indentado (las líneas dentro del with)
* Cuando el bloque termina — ya sea con éxito o por un error — llama automáticamente a file.close()

Nota: `encoding='utf-8'` le indica a Python que lea los caracteres como si estuvieran codificados en el formato UTF-8. <BR> 
UTF-8 (Unicode Transformation Format – 8 bits) es el estándar más usado actualmente en todo el mundo.
Permite representar letras acentuadas, eñes, símbolos, emojis, y caracteres de otros idiomas (como chino o japonés).

# Escritura de archvios

In [39]:
# Escritura de un archivo
try:
    with open("data/notas.txt", "w") as file:
        file.write("Alumno: Ana\n")
        file.write("Nota: 8\n")
except Exception as e:
    print(e)

    

In [40]:
# Leemos a ver que cambió
try:
    with open("data/notas.txt", "r") as file:
        contenido = file.read()
        print(contenido)
except Exception as e:
    print(e)

    

Alumno: Ana
Nota: 8



Veamos algunos casos posibles de escritura de archivos, contenido, métodos y formato de archivos

## Caso 1 - Lista de listas

In [106]:
# Proponemos la siguiente lista de listas
estudiantes = [
    [1, "Thiago", "Almada", 19],
    [2, "Agostina", "Hein", 25],
    [3, "Leandro", "Bolmaro", 22]
]

header = ["id", "nombre", "apellido"]

### Usando formato txt

In [63]:
# Escribimos la lista de estudiantes en el archivo txt_estudiantes.txt
with open("data/txt_estudiantes.txt", "w", encoding="utf-8") as file:
    for e in estudiantes:
        # file.write(str(e)+"\n")
        file.write(f"{e[0]}, {e[1]}, {e[2]}, {e[3]}\n")



In [64]:
# Leemos y mostramos el contenido
with open("data/txt_estudiantes.txt", "r", encoding="utf-8") as file:
    contenido = file.read()
    print(contenido)

1, Thiago, Almada, 19
2, Agostina, Hein, 25
3, Leandro, Bolmaro, 22
4, Valentina, Raposo, 24



Ventajas:
* Simple de crear y leer manualmente.
* Ideal para texto sin estructura compleja.
<BR>

Desventajas:
* No tiene encabezados.
* No hay estructura formal (cada línea hay que “parsearla” al leerla).

### Usando formato csv

In [112]:
import csv

with open("data/csv_estudiantes.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(header)  # encabezado
    writer.writerows(estudiantes) # body


In [114]:
# Leemos y mostramos el contenido
with open("data/csv_estudiantes.csv", "r", encoding="utf-8") as file:
    contenido = csv.reader(file)
    for fila in contenido:
        print(fila)

['id', 'nombre', 'apellido']
['1', 'Thiago', 'Almada', '19']
['2', 'Agostina', 'Hein', '25']
['3', 'Leandro', 'Bolmaro', '22']
['4', 'Valentina', 'Raposo']


In [113]:
# Recordemos que también podemos insertar una row al final
with open("data/csv_estudiantes.csv", "a", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow([4, "Valentina", "Raposo"]) # body

Ventajas:
* Formato estándar e intercambiable (Excel, Google Sheets, Pandas).
* Fácil de leer y escribir con el módulo csv.
* Ideal para listas de listas.

<BR>
Desventajas:
* Solo almacena texto (no tipos complejos como listas anidadas o diccionarios).

## Caso 2 - Lista de diccionarios

In [150]:
estudiantes = [
    {"id": 1, "nombre": "Thiago", "apellido": "Almada"},
    {"id": 2, "nombre": "Agostina", "apellido": "Hein"},
    {"id": 3, "nombre": "Leandro", "apellido": "Bolmaro"},
    {"id": 4, "nombre": "Valentina", "apellido": "Raposo"},
]

### Usando formato json

In [135]:
import json

# Crear y escribir el archivo JSON
with open("data/estudiantes.json", "w", encoding="utf-8") as f:
    json.dump(estudiantes, f, indent=3, ensure_ascii=False)

In [136]:
import json

# Abrir y leer el archivo JSON
with open("data/estudiantes.json", "r", encoding="utf-8") as f:
    datos = json.load(f)
    for e in datos:    
        print(e)

{'id': 1, 'nombre': 'Thiago', 'apellido': 'Almada'}
{'id': 2, 'nombre': 'Agostina', 'apellido': 'Hein'}
{'id': 3, 'nombre': 'Leandro', 'apellido': 'Bolmaro'}
{'id': 4, 'nombre': 'Valentina', 'apellido': 'Raposo'}


`ensure_ascii=True`	Convierte todos los caracteres fuera del ASCII (0–127) a secuencias \uXXXX.
<BR>

`ensure_ascii=False` Mantiene los caracteres originales tal como están (recomendado si usás encoding="utf-8").

In [138]:
clientes = [
    {"nombre": "Oscar", "emoji": "😊"}
]

In [149]:
import json

# Crear y escribir el archivo JSON
with open("data/clientes.json", "w", encoding="utf-8") as f:
    json.dump(clientes, f, indent=3, ensure_ascii=False)

In [147]:
# Abrir y leer el archivo JSON
with open("data/clientes.json", "r", encoding="utf-8") as f:
    datos = json.load(f)
    for e in datos:    
        print(e)

{'nombre': 'Oscar', 'emoji': '😊'}


Recomendación:
<BR>

* Archivos JSON con texto en español (ñ, á, é, í, ó, ú) => ensure_ascii=False
* Comunicación con APIs que solo aceptan ASCII puro => ensure_ascii=True
* Guardar datos legibles por humanos => ensure_ascii=False